In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_backfill as pb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl
import bar_meta as bm
import filters as ft
# import market_cluster as mc

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# from tqdm import tqdm
# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')

# .plot_bokeh(sizing_mode="scale_height")

In [9]:
symbol='GLD'
# symbol = 'market'
# date = '2020-01-02'
start_date = '2020-01-01'
# start_date = '2020-10-01'
# end_date = '2020-01-10'
# symbols=['SPY', 'QQQ']
tick_type='trades'

result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

thresh = {
    # GT thresholds
    'renko_reveral_multiple': 2,
    'duration_sec': 60*45,
    # LT thresholds 
    'min_duration_sec': 10,
    'min_tick_count': 30,
}

In [10]:
daily_vol_df = ps3.get_symbol_vol_filter(result_path, symbol, start_date)
daily_vol_df.loc[:, 'range_jma_lag'] = daily_vol_df.range_jma.shift(1)
daily_vol_df = daily_vol_df.dropna()


In [ ]:
bar_dates = bm.build_bars_dates_ray(daily_vol_df, thresh, result_path, symbol)

In [22]:
daily_bar_stats_df = bm.process_bar_dates(bar_dates)

In [23]:
daily_join_df = pd.merge(left=daily_bar_stats_df, right=daily_vol_df, left_on='date', right_on='date')

In [ ]:
bar_dates_2 = bm.build_bars_dates_ray(daily_join_df, thresh, result_path, symbol)

2020-10-27 00:35:42,455	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


(pid=93313) GLD 2020-01-06 trying to get ticks from local file...
(pid=93312) GLD 2020-01-07 trying to get ticks from local file...
(pid=93311) GLD 2020-01-08 trying to get ticks from local file...
(pid=93314) GLD 2020-01-09 trying to get ticks from local file...
(pid=93327) GLD 2020-01-10 trying to get ticks from local file...
(pid=93312) GLD 2020-01-13 trying to get ticks from local file...
(pid=93313) GLD 2020-01-14 trying to get ticks from local file...
(pid=93327) GLD 2020-01-15 trying to get ticks from local file...
(pid=93312) GLD 2020-01-16 trying to get ticks from local file...
(pid=93313) GLD 2020-01-17 trying to get ticks from local file...
(pid=93327) GLD 2020-01-21 trying to get ticks from local file...
(pid=93311) GLD 2020-01-22 trying to get ticks from local file...
(pid=93312) GLD 2020-01-23 trying to get ticks from local file...
(pid=93313) GLD 2020-01-24 trying to get ticks from local file...
(pid=93311) GLD 2020-01-27 trying to get ticks from local file...
(pid=93327

In [ ]:
bars_df = pd.DataFrame(bar_dates_2[0]['bars'])
bars_df = pd.DataFrame(bar_dates[0]['bars'])

In [ ]:
abs(bars_df.tick_imbalance).describe()

In [ ]:
pd.DataFrame(bar_dates_3[130]['bars']).bar_trigger.value_counts()

In [ ]:
pd.Series([len(date['bars']) for date in bar_dates_3]).describe()

In [ ]:
pd.Series([len(date['bars']) for date in bar_dates_2]).describe()

In [ ]:
pd.Series([len(date['bars']) for date in bar_dates]).describe()

In [ ]:
results = []
for date_d in bar_dates:
    
    imbalances = []    
    for bar in date_d['bars']:
        imbalances.append(bar['tick_imbalance'])
    
    imbal_q80 = pd.Series(imbalances).quantile(q=.8)
    results.append({'date': date_d['date'], 'bar_count': len(date_d['bars']), 'tick_imbalance_p80': imbal_q80})


In [ ]:
show_grid(pd.DataFrame(bar_dates[open_dates[2]])[['bar_trigger','open_at','close_at','duration_min','ticks','price_wmean']])

In [ ]:
# range / price = HFT profit potential
(df.price.max() - df.price.min()) / df.price.mean()

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

lbars_unq = bl.get_concurrent_stats(lbars_df)
print(lbars_unq['grand_avg_unq'])

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])

In [ ]:
df[['range', 'range_jma']].plot()
# df2[['range', 'range_jma']].plot_bokeh(sizing_mode="scale_height")